In [1]:
import numpy as np
import tensorflow as tf

In [25]:
# # original model
# # imagePath = './Report/AltraDifficult01.jpg'
# # imagePath = './Report/AltraOccluded.jpg'
# # imagePath = './Report/NikeMorphed.jpg'
# # imagePath = './Report/NikeMorphedCrop.jpg'
# # imagePath = './Report/NikeDifficult01.jpg'
# # imagePath = './Report/NikeAltra.jpg'
# imagePath = './Report/Neither.jpg'
# modelFullPath = './Report/original_output_graph.pb'
# labelsFullPath = './Report/original_output_labels.txt'

In [20]:
# newer model
# imagePath = './Report/AltraDifficult01.jpg'
# imagePath = './Report/AltraOccluded.jpg'
# imagePath = './Report/NikeAltra.jpg'
# imagePath = './Report/NikeMorphedCrop.jpg'
# imagePath = './Report/NikeDifficult01.jpg'
imagePath = './Report/Neither.jpg'
modelFullPath = './Report/output_graph.pb'
labelsFullPath = './Report/output_labels.txt'

In [21]:
answer = None

if not tf.gfile.Exists(imagePath):
    tf.logging.fatal('File does not exist %s', imagePath)
    #return answer

image_data = tf.gfile.FastGFile(imagePath, 'rb').read()

In [22]:
with tf.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0': image_data})
    predictions = np.squeeze(predictions)
    top_k = predictions.argsort()[-5:][::-1]  # Getting top 5 predictions
    f = open(labelsFullPath, 'rb')
    lines = f.readlines()
    labels = [str(w).replace("\n", "") for w in lines]
    for node_id in top_k:
        human_string = labels[node_id]
        score = predictions[node_id]
        print('%s (score = %.5f)' % (human_string, score))

    answer = labels[top_k[0]]

neither (score = 0.85841)
nike (score = 0.13508)
altra (score = 0.00651)


In [3]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [4]:
# Creates graph from saved GraphDef.
create_graph()